# Dashboard for GVOs (Geomagnetic Virtual Observatories)

Initial prototype before integration into VirES server

Product documentation (check links at the right):  
https://www.space.dtu.dk/english/research/projects/project-descriptions/geomagnetic-virtual-observatories

For this notebook, put the file `SW_OPER_VOBS_1M_2_20131215T000000_20200315T000000_0102.cdf` into the running directory, obtained from:  
http://www.spacecenter.dk/files/magnetic-models/GVO/GVO_data_SWARM.zip

In [ ]:
!pip install --upgrade 'git+https://github.com/ESA-VirES/VirES-Python-Client@swarm_gvo#egg=viresclient'

In [ ]:
import datetime as dt
import numpy as np
import cdflib
import xarray as xr
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from viresclient import SwarmRequest

import panel as pn
import hvplot.xarray
import holoviews as hv
import geoviews as gv
# import geoviews.feature as gf
hv.extension('bokeh', 'matplotlib')
# gv.extension('bokeh', 'matplotlib')
pn.extension()

## Load in the GVO dataset

In [ ]:
def fetch_GVO():
    request = SwarmRequest('https://staging.viresdisc.vires.services/ows')
    request.set_collection('SW_OPER_VOBS_1M_2_')
    request.set_products(
        measurements=['SiteCode', 'B_CF', 'B_OB', 'sigma_CF', 'sigma_OB'],
        models=["'CHAOS-Core' = 'CHAOS-Core'(max_degree=14)"]
    )
    data = request.get_between(
        dt.datetime(2000, 1, 1),
        dt.datetime(2021, 1, 1),
        asynchronous=False, show_progress=False
    )
    ds = data.as_xarray(reshape=True)
    return ds

ds = fetch_GVO()
ds

In [ ]:
# # Now we can easily slice out data from specific GVOs:
# ds.sel(NEC="C", Site=0)["B_OB"].plot.line(x="Timestamp")

## Plot with cartopy/matplotlib

In [ ]:
GVO_LOCATIONS = np.vstack((ds["Longitude"].values, ds["Latitude"].values)).T

LINE_COLORS = {
    "C": mpl.colors.to_hex("tab:blue"),
    "N": mpl.colors.to_hex("tab:orange"),
    "E": mpl.colors.to_hex("tab:green"),
    "Model": mpl.colors.to_hex("tab:grey"),
}

In [ ]:
def make_map(NEC="C", highlight_Site=0):
    color = LINE_COLORS.get(NEC)
    fig = plt.figure(figsize=(8,4))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree(),extent=[-180, 180, -90, 90])
    ax.coastlines()
    ax.scatter(*GVO_LOCATIONS.T, color="black", s=3, zorder=2)
#     mini = ds.sel(NEC="C")["B_OB"].data.min()
#     maxi = ds.sel(NEC="C")["B_OB"].data.max()
    # Loop through each GVO
    for i in range(300):
        gvo_record = ds["B_OB"].sel(NEC=NEC, Site=i)
        # Extract a fractional (0-1) representation of the GVO data
#         gvo_y_scaled = (gvo_record.data - mini) / (maxi - mini)
        # SCALED SEPARATELY FOR EACH GVO
        gvo_y_scaled = (gvo_record.data - np.nanmin(gvo_record.data)) / (np.nanmax(gvo_record.data) - np.nanmin(gvo_record.data))
        # Construct vertices to represent the GVO data, scaled between -5 and 5
        # And centred around the GVO location in lat/lon space
        lat, lon = float(gvo_record.Latitude), float(gvo_record.Longitude)
        xdata = lon + np.linspace(-5, 5, len(gvo_y_scaled))
        ydata = lat + 10*(gvo_y_scaled - 0.5)
        gvo_xy_verts = np.vstack((xdata, ydata))
        # Plot these lines onto the figure
        ax.plot(*gvo_xy_verts, transform=ccrs.PlateCarree(), color=color, alpha=0.8, linewidth=0.8)
    # Draw a circle around selected grid point
    Site_latlon = ds[["Latitude", "Longitude"]].sel(Site=highlight_Site)
    lat, lon = float(Site_latlon["Latitude"]), float(Site_latlon["Longitude"])
    ax.scatter(
        lon, lat, transform=ccrs.PlateCarree(),
        s=350, facecolors='grey', edgecolors='grey', linewidth=5
    )
    fig.text(0.5, 1, f"Selected LAT={round(lat, 1)}, LON={round(lon, 1)}", fontsize=15, ha="center", va="top")
    return fig

# make_map(NEC="C", highlight_Site=101);

## Redo map plot with GeoViews

In [ ]:
def wee_lines_for_map(ds, NEC="C", paths_or_points="points", param="OB"):
    """Returns GeoViews Paths for all the little lines to go on the map"""
    parameter = f"B_{param}"
    wee_paths = []
    lons = []
    lats = []
    for Site in range(300):
        gvo_record = ds[parameter].sel(NEC=NEC, Site=Site)
        if all(np.isnan(gvo_record.data)):
            continue
        # Extract a fractional (0-1) representation of the GVO data
        #         gvo_y_scaled = (gvo_record.data - mini) / (maxi - mini)
        # SCALED SEPARATELY FOR EACH GVO
        # subsamble the data to make it a bit lighter
        mini, maxi = np.nanmin(gvo_record.data), np.nanmax(gvo_record.data)
#         data = gvo_record.data[::4]
        data = gvo_record.data
        gvo_y_scaled = (data - mini) / (maxi - mini)
        # Screen out nans from this (they break the plotting)
        gvo_y_scaled = gvo_y_scaled[~np.isnan(gvo_y_scaled)]
        # Construct vertices to represent the GVO data, scaled between -5 and 5
        # And centred around the GVO location in lat/lon space
        lat, lon = float(gvo_record.Latitude), float(gvo_record.Longitude)
        xdata = lon + np.linspace(-5, 5, len(gvo_y_scaled))
        ydata = lat + 10*(gvo_y_scaled - 0.5)
        lons.append(xdata)
        lats.append(ydata)
    if paths_or_points=="points":
        # Merge all points from all GVOs to plot as one overlay
        lons = np.concatenate(lons)
        lats = np.concatenate(lats)
        lonlats = np.vstack((lons, lats)).T
        return gv.Overlay([gv.Points(lonlats, ["Longitude", "Latitude"]).opts(color=LINE_COLORS[NEC], size=0.5, alpha=1)], group="GVO_data")
    elif paths_or_points=="paths":
        # Plot data as paths, from each GVO individually (Slow)
        gvo_xy_verts_all = [dict(Longitude=xdata, Latitude=ydata) for xdata, ydata in zip(lons, lats)]
        wee_paths = [
            gv.Path(gvo_xy_verts).opts(color=LINE_COLORS[NEC])
            for gvo_xy_verts in gvo_xy_verts_all
        ]
        return gv.Overlay(wee_paths, group="GVO_data")


class GVOMap():
    
    def __init__(self, NEC="C", draw_lines=True, use_pointer=True, param="OB"):
        self.basemap = gv.Overlay(self._basemap_with_GVO_points())
        if use_pointer:
            self.pointer_tap = hv.streams.Tap(source=self.basemap, x=0, y=90)
            self.basemap *= self._highlight_gvo()
        self.update_figure_with_lines(NEC=NEC, draw_lines=draw_lines, param=param)
        # Set the pointer_tap as a property of the hv object
        # so we can easily extract current position with p.object.pointer_tap
        if use_pointer:
            self.object.pointer_tap = self.pointer_tap
    
    def update_figure_with_lines(self, NEC="C", draw_lines=True, param="OB"):
        """Provides an updatable figure accessible at self.object"""
        if draw_lines:
            self.object = self.basemap * wee_lines_for_map(ds, NEC=NEC, param=param)
        else:
            self.object = self.basemap
        self.object.opts(
            projection=ccrs.PlateCarree(), aspect=2, global_extent=True, frame_width=500,
#             default_tools=[], tools=[], active_tools=[]  # why doesn't this work?
        )
    
    def _basemap_with_GVO_points(self):
        """Generates list of static GeoViews elements for the underlying map"""
        elements = [
            gv.feature.ocean.opts(alpha=0.7),
            gv.feature.land.opts(alpha=0.7),
        ]
        # Dots for the centres of all GVOs
        elements.append(
            gv.Points(GVO_LOCATIONS).opts(color="black", size=1)
#             .opts(default_tools=[], tools=[], active_tools=[])
        )
        return elements
    
    def _highlight_gvo(self):
        return gv.DynamicMap(
            lambda x, y: gv.Points([self.nearest_GVO(x, y)[:2]]),
            streams=[self.pointer_tap],
        ).opts(projection=ccrs.PlateCarree(), color="grey", size=18, alpha=0.8)

    @staticmethod
    def nearest_GVO(x, y):
        """Given (x, y) location, return (x, y, Site) of the nearest GVO"""
        Site = int(np.argmin(np.sum((GVO_LOCATIONS - np.array([x, y]))**2, axis=1)))
        x, y = GVO_LOCATIONS[Site]
        return x, y, Site
    
    def selected_GVO_Site(self):
        x, y = self.pointer_tap.x, self.pointer_tap.y
        _, _, Site = self.nearest_GVO(x, y)
        return Site

# p1 = GVOMap(draw_lines=True, use_pointer=True, NEC="C")
# p1.update_figure_with_lines("E", param="OB")
# p1.basemap + p1.object

In [ ]:
# print(
#     p1.object.pointer_tap, "\n"
#     "Nearest GVO: ",
#     GVOMap.nearest_GVO(p1.object.pointer_tap.x, p1.object.pointer_tap.y),
# )

In [ ]:
# html = pn.pane.HTML("Clicked: ...")

# # # # watch for changes with .param.watch
# def f(eventx, eventy):
#     x, y = eventx.obj.x, eventx.obj.y
#     x, y, Site = GVOMap.nearest_GVO(x, y)
#     html.object = f"Clicked: x:{x}, y:{y}, Site:{Site}"
# p1.object.pointer_tap.param.watch(f, ["x", "y"])

# html

## HoloViews plot for individual GVO

In [ ]:
def make_gvo_lineplot(Site=0, param="OB", model="CHAOS-Core"):
    """Generate holoviews object showing the three components"""
    
    def _select_obs_and_rename_vars(ds, Site, param):
        # Must split apart and rename components so that they are treated as
        # separate entities with shared_axes in hvplot
        # https://discourse.holoviz.org/t/how-to-only-link-share-only-the-x-axis/93
        _ds = ds[[f"B_{param}", f"sigma_{param}", f"B_NEC_{model}"]]
        _ds = _ds.sel(Site=Site)
        _ds[f"B_{param} (C)"] = _ds[f"B_{param}"].sel(NEC="C")
        _ds[f"B_{param} (N)"] = _ds[f"B_{param}"].sel(NEC="N")
        _ds[f"B_{param} (E)"] = _ds[f"B_{param}"].sel(NEC="E")
        _ds[f"sigma_{param} (C)"] = _ds[f"sigma_{param}"].sel(NEC="C")
        _ds[f"sigma_{param} (N)"] = _ds[f"sigma_{param}"].sel(NEC="N")
        _ds[f"sigma_{param} (E)"] = _ds[f"sigma_{param}"].sel(NEC="E")
        _ds[f"B_NEC_{model} (C)"] = _ds[f"B_NEC_{model}"].sel(NEC="C")
        _ds[f"B_NEC_{model} (N)"] = _ds[f"B_NEC_{model}"].sel(NEC="N")
        _ds[f"B_NEC_{model} (E)"] = _ds[f"B_NEC_{model}"].sel(NEC="E")
#         _ds = _ds.drop_vars(["B_OB", "sigma_OB"])
        _ds = _ds.drop_dims("NEC")
        return _ds

    _ds = _select_obs_and_rename_vars(ds, Site, param)
    color_r = LINE_COLORS["C"]
    color_t = LINE_COLORS["N"]
    color_p = LINE_COLORS["E"]
    color_model = LINE_COLORS["Model"]
    lineplots = \
          hvplot.plot(_ds, kind="line", x="Timestamp", y=f"B_{param} (C)", line_color=color_r, xlabel="", grid=True, height=180, width=500) \
            * hvplot.plot(_ds, kind="errorbars", x="Timestamp", y=f"B_{param} (C)", yerr1=f"sigma_{param} (C)") \
            * hvplot.plot(_ds, kind="line", x="Timestamp", y=f"B_NEC_{model} (C)", line_color=color_model) \
        + hvplot.plot(_ds, kind="line", x="Timestamp", y=f"B_{param} (N)", line_color=color_t, xlabel="", grid=True, height=180, width=500) \
            * hvplot.plot(_ds, kind="errorbars", x="Timestamp", y=f"B_{param} (N)", yerr1=f"sigma_{param} (N)") \
            * hvplot.plot(_ds, kind="line", x="Timestamp", y=f"B_NEC_{model} (N)", line_color=color_model) \
        + hvplot.plot(_ds, kind="line", x="Timestamp", y=f"B_{param} (E)", line_color=color_p, xlabel="", grid=True, height=180, width=500) \
            * hvplot.plot(_ds, kind="errorbars", x="Timestamp", y=f"B_{param} (E)", yerr1=f"sigma_{param} (E)") \
            * hvplot.plot(_ds, kind="line", x="Timestamp", y=f"B_NEC_{model} (E)", line_color=color_model)
    lat = float(_ds.Latitude)
    lon = float(_ds.Longitude)
    title = f"Selected LAT={round(lat, 1)}, LON={round(lon, 1)}"
    lineplots.cols(1).opts(shared_axes=True, sizing_mode="stretch_both", title=title)
    return lineplots

# p = make_gvo_lineplot(Site=30, param="OB")
# p

## Build dashboard with Panel

In [ ]:
panes = {
    "hv-map": pn.pane.HoloViews(),
    "hv-gvo-line": pn.pane.HoloViews()
}

widgets = {
    "select_cpt": pn.widgets.Select(name="Preview component on map:", options=["C", "N", "E"], width=200),
    "select_param": pn.widgets.Select(name="Select parameter:", options=["OB", "CF"], width=200)
}

loading_indicator = pn.pane.HTML("")
# pn.indicators.LoadingSpinner(value=False, width=50, height=50)
def set_loading(loading):
    if loading:
        loading_indicator.object = '<p style="color:red; font-size:200%"><strong>Loading...</strong></p>'
    else:
        loading_indicator.object = ''

# Map with tap to select GVO
hv_map = GVOMap(NEC="C", draw_lines=True, use_pointer=True, param="OB")
panes["hv-map"].object = hv_map.object
def update_map_NEC(event):
    """Change the selected component shown on the map"""
    set_loading(True)
    hv_map.update_figure_with_lines(NEC=event.obj.value, param=widgets["select_param"].value)
    panes["hv-map"].object = hv_map.object
    set_loading(False)
def update_map_param(event):
    """Change the selected parameter shown on the map"""
    set_loading(True)
    param = event.obj.value
    hv_map.update_figure_with_lines(NEC=widgets["select_cpt"].value, param=param)
    panes["hv-map"].object = hv_map.object
    # Also update the GVO line plots
    Site = hv_map.selected_GVO_Site()
    panes["hv-gvo-line"].object = make_gvo_lineplot(Site, param=param)
    set_loading(False)

# Detailed line plots from GVOs
panes["hv-gvo-line"].object = make_gvo_lineplot(0)
def update_gvo_lines(event):
    """Update the line figure according to the pointer tap on the map"""
    set_loading(True)
    x, y = event.obj.x, event.obj.y
    x, y, Site = GVOMap.nearest_GVO(x, y)
    panes["hv-gvo-line"].object = make_gvo_lineplot(Site, param=widgets["select_param"].value)
    set_loading(False)

hv_map.object.pointer_tap.param.watch(update_gvo_lines, "x")
widgets["select_cpt"].param.watch(update_map_NEC, "value")
widgets["select_param"].param.watch(update_map_param, "value")

intro_text = """
Notes:\n
- currently displaying the SW_OPER_VOBS_1M_2_ dataset (Swarm 1-monthly series)
- OB = observed total field; CF = core field estimate after denoising
- NEC = (North, East, Centre) coords (as they are output from VirES, being consistent with other Swarm datasets)
- probably will alter them here to instead display (radial,theta,phi)?
- grey lines: CHAOS-Core(max_degree=14) predictions
"""

dashboard = pn.Row(
    pn.Column(
        intro_text,
        widgets["select_param"],
        "Click on the map to select a GVO:",
        panes["hv-map"],
        pn.Row(widgets["select_cpt"], loading_indicator),
    ),
    pn.Column(panes["hv-gvo-line"], width=400),
)

In [ ]:
dashboard.servable()

---

TODO:

- add options to select each GVO dataset, and select different models
- get ideas for advanced interactivity (e.g. fitting SH models)
- eventually serve as standalone dashboard:
    - https://panel.holoviz.org/user_guide/Deploy_and_Export.html
    - can be done for an individual user with `panel serve <notebook.ipynb>`
    - example from docker container:
      `docker run --rm -it -u jovyan -p 5006:5006 -v ~/Temp/vo-dashboard:/home/jovyan registry.gitlab.eox.at/esa/vires_vre_ops/vre-swarm-notebook:0.7.5 panel serve vo-dashboard.ipynb`